# Reproduction Table 1, QCNN
In this notebook the QCNN result of table 1 is reproduced using PyTorch.

In [ ]:
# run this cell to download the right packages (only needed once)
!python --version

!pip install cifar10
!pip install imageio numpy scipy    
!pip install git+https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks

Python 3.9.16
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to /tmp/pip-req-build-mnqemws1
  Running command git clone --filter=blob:none --quiet https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks /tmp/pip-req-build-mnqemws1
  Resolved https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to commit 28caa7cde240e354fd7b87280450fd233cd494c3
  Preparing metadata (setup.py) ... done
  Created wheel for Pytorch-QNN: filename=Pytorch_QNN-1-py3-none-any.whl size=21507 sha256=2806e9054a3dbad45c88447967f5d62db109914f0e3c1e0873a2b94404fb6476
  Stored in directory: /tmp/pip-ephem-wheel-cache-_uw_7aa0/wheels/4d/ef/23/2dab0a09f3d2ba797d554a61

In [ ]:
import torch
import numpy as np
import tensorflow as tf

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from pathlib import Path
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision import datasets, transforms

from core_qnn.quaternion_layers import QuaternionConv, QuaternionLinear
from core_qnn.quaternion_ops import check_input, q_normalize

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
%%time

# import and download the CIFAR10 dataset
batch_size = 32

transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5),(.5,.5,.5))])
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
CPU times: user 3.32 s, sys: 963 ms, total: 4.28 s
Wall time: 5.89 s


In [ ]:
%%time

class QCNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_features, kernel_size):
        super(QCNN, self).__init__()

        self.conv_1 = QuaternionConv(in_channels, hidden_channels[0], kernel_size=kernel_size, stride=1)
        self.conv_2 = QuaternionConv(hidden_channels[0], hidden_channels[1], kernel_size=kernel_size, stride=1)

        self.pool_1 = nn.MaxPool2d(2, 2)
        self.dropout_1 = nn.Dropout(0.25)

        self.conv_3 = QuaternionConv(hidden_channels[1], hidden_channels[2], kernel_size=kernel_size, stride=1)
        self.conv_4 = QuaternionConv(hidden_channels[2], hidden_channels[3], kernel_size=kernel_size, stride=1)

        self.pool_2 = nn.MaxPool2d(2, 2)
        self.dropout_2 = nn.Dropout(0.25)

        self.fc_1 = QuaternionLinear(12800, 512)
        self.fc_2 = nn.Linear(512, out_features)

        self.dropout_3 = nn.Dropout(0.5)
        self.sm = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool_1(x)
        x = self.dropout_1(x)

        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = self.pool_2(x)
        x = self.dropout_2(x)

        x = torch.flatten(x, start_dim=1) 

        x = F.relu(self.fc_1(x))
        x = self.dropout_3(x)
        x = self.fc_2(x)
        x = self.sm(x)

        return x

# Model parameters
in_channels = 4
hidden_channels = [64, 128, 256, 512]
out_features = 10
kernel_size = (3, 3)

qcnn = QCNN(in_channels, hidden_channels, out_features, kernel_size)
qcnn = qcnn.cuda()

print("Number of trainable parameters: ", sum(p.numel() for p in qcnn.parameters() if p.requires_grad))

summary(qcnn, input_size=(in_channels, 32, 32), batch_size=batch_size, device=device.type)

Number of trainable parameters:  2032650
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    QuaternionConv-1           [32, 64, 30, 30]              64
    QuaternionConv-2          [32, 128, 28, 28]             128
         MaxPool2d-3          [32, 128, 14, 14]               0
           Dropout-4          [32, 128, 14, 14]               0
    QuaternionConv-5          [32, 256, 12, 12]             256
    QuaternionConv-6          [32, 512, 10, 10]             512
         MaxPool2d-7            [32, 512, 5, 5]               0
           Dropout-8            [32, 512, 5, 5]               0
  QuaternionLinear-9                  [32, 512]             512
          Dropout-10                  [32, 512]               0
           Linear-11                   [32, 10]           5,130
          Softmax-12                   [32, 10]               0
Total params: 6,602
Trainable params: 5,130
Non-trainable para

In [ ]:
%%time
num_epochs = 80
learning_rate = 0.0001
learning_rate_decay = 1e-6

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(qcnn.parameters(),lr=learning_rate, weight_decay=learning_rate_decay)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  
  qcnn.train()

  for index, (x_batch, y_batch) in enumerate(train_loader):
    zeros_channel = torch.zeros((x_batch.shape[0], 1, x_batch.shape[2], x_batch.shape[3]))
    x_batch = torch.cat([x_batch, zeros_channel], dim=1)

    # Check if the input size is correct
    check_input(x_batch)

    x_batch = x_batch.cuda()
    y_batch = y_batch.cuda()
    
    # Perform forward pass
    y_pred = qcnn(x_batch)

    # Compute the loss
    loss = criterion(y_pred, y_batch)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print (f'Epoch [{epoch + 1}/{num_epochs}], Last loss: {loss.item():.4f}')

print("Finished training")

Epoch [1/80], Last loss: 1.9886
Epoch [2/80], Last loss: 1.8697
Epoch [3/80], Last loss: 1.9628
Epoch [4/80], Last loss: 2.1272
Epoch [5/80], Last loss: 1.8904
Epoch [6/80], Last loss: 1.8214
Epoch [7/80], Last loss: 1.8542
Epoch [8/80], Last loss: 2.2121
Epoch [9/80], Last loss: 1.6685
Epoch [10/80], Last loss: 1.9833
Epoch [11/80], Last loss: 1.9128
Epoch [12/80], Last loss: 1.7872
Epoch [13/80], Last loss: 1.5169
Epoch [14/80], Last loss: 1.8736
Epoch [15/80], Last loss: 1.8766
Epoch [16/80], Last loss: 1.8785
Epoch [17/80], Last loss: 1.6499
Epoch [18/80], Last loss: 1.6673
Epoch [19/80], Last loss: 1.6441
Epoch [20/80], Last loss: 1.6456
Epoch [21/80], Last loss: 1.6943
Epoch [22/80], Last loss: 1.7390
Epoch [23/80], Last loss: 1.7184
Epoch [24/80], Last loss: 1.6462
Epoch [25/80], Last loss: 1.7753
Epoch [26/80], Last loss: 1.5445
Epoch [27/80], Last loss: 1.8330
Epoch [28/80], Last loss: 1.6092
Epoch [29/80], Last loss: 1.5331
Epoch [30/80], Last loss: 1.7576
Epoch [31/80], Last

In [ ]:
%%time

with torch.no_grad():
    n_correct = 0
    n_samples = 0

    qcnn.eval()

    for index, (x_batch, y_batch) in enumerate(test_loader):
      zeros_channel = torch.zeros((x_batch.shape[0], 1, x_batch.shape[2], x_batch.shape[3]))
      x_batch = torch.cat([x_batch, zeros_channel], dim=1)

      x_batch = x_batch.cuda()
      y_batch = y_batch.cuda()

      # Check if the input size is correct
      check_input(x_batch)

      # Perform forward pass
      y_pred = qcnn(x_batch)

      _, predicted = torch.max(y_pred,1)
      n_samples += y_batch.size(0)
      n_correct += (predicted == y_batch).sum().item()

    acc = 100 * n_correct / n_samples
    print(f"\r\n Accuracy of the network: {acc}% \r\n")



 Accuracy of the network: 77.05% 

CPU times: user 1.53 s, sys: 219 ms, total: 1.75 s
Wall time: 3.25 s
